# Amazon-Product-Recommendation-System :- Top Recommendations

In [1]:
import pandas as pd
import numpy as np
import joblib

# Load trained factors and mappings
user_factors = joblib.load("../model/user_factors.npy")
item_factors = joblib.load("../model/item_factors.npy")
user_map = joblib.load("../model/user_map.pkl")
item_map = joblib.load("../model/item_map.pkl")

# Load cleaned dataset
df = pd.read_csv(r"C:\Users\giris\Documents\datasets for practice\cleaned_amazon_reviews.csv")

In [2]:
# Convert mapping to ID → index
user_index = {v: k for k, v in user_map.items()}
item_index = {v: k for k, v in item_map.items()}

# Map dataset IDs to model indices
df = df[
    df['UserId'].isin(user_index) &
    df['ProductId'].isin(item_index)
]

df['user_idx'] = df['UserId'].map(user_index)
df['item_idx'] = df['ProductId'].map(item_index)


In [3]:
def predict_rating(user_idx, item_idx):
    return np.dot(user_factors[user_idx], item_factors[item_idx])

In [4]:
import random

user_id = random.choice(df['UserId'].unique())
user_idx = user_index[user_id]

print("Testing recommendations for user:", user_id)


Testing recommendations for user: A3EXIEHNL8UZCZ


In [5]:
# Items already rated by the user
seen = set(df[df['user_idx'] == user_idx]['item_idx'])

# Predict for unseen items
scores = []

for i in range(item_factors.shape[0]):
    if i not in seen:
        scores.append((i, predict_rating(user_idx, i)))

# Sort by predicted rating
scores = sorted(scores, key=lambda x: x[1], reverse=True)

# Top N items
TOP_N = 5
top_items = scores[:TOP_N]

# Convert indices → ProductIds
recommended_products = [item_map[i] for i, _ in top_items]

recommended_products


['B000ETC9I8', 'B000LKYS66', 'B0018Q7OE0', 'B002KADFZU', 'B000LKTKIC']

In [6]:
for idx, score in top_items:
    print(item_map[idx], round(score, 2))

B000ETC9I8 0.14
B000LKYS66 0.14
B0018Q7OE0 0.14
B002KADFZU 0.14
B000LKTKIC 0.13
